In [ ]:
import numpy as np
import pandas as pd 
import os

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from scipy import stats

le = LabelEncoder()

ss = pd.read_csv('../input/tabular-playground-series-feb-2022/sample_submission.csv')
sub2 = pd.read_csv('../input/simple-k-neighbors/output.csv')
sub3 = pd.read_csv('../input/tps-feb-resnet-my-version/submission.csv') 
sub4 = pd.read_csv('../input/tpsfeb22-soft-voting-baseline/submission.csv') 


In [ ]:
sub2['target'] = le.fit_transform(sub2['target'])
sub3['target'] = le.transform(sub3['target'])
sub4['target'] = le.transform(sub4['target'])

blend_preds = []
for preds in [sub2, sub3, sub4]:
    blend_preds.append(preds['target'])

## Check three different values

In [ ]:
count = 0
for i in range(len(blend_preds[0])):
    if (blend_preds[0][i] != blend_preds[1][i]) and (blend_preds[1][i] != blend_preds[2][i]) and (blend_preds[0][i] != blend_preds[2][i]):
        print(f"Index: {i}, tabular-playground-series-feb-2022: {blend_preds[0][i]}, simple-k-neighbors: {blend_preds[1][i]}, tpsfeb22-soft-voting-baseline: {blend_preds[2][i]} \n")
        count += 1
print(f"count: {count}")

### TODO: We need to think about what we can do with this knowledge

In [ ]:
blend_ss = ss.copy()
blend_ss['target'] = le.inverse_transform(stats.mode(np.column_stack(blend_preds), axis=1)[0])

blend_ss

In [ ]:
np.unique(blend_ss['target'], return_counts=True)

In [ ]:
blend_ss.target.value_counts()

In [ ]:
sub = blend_ss

## Check the intersection between train & test dataset

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-feb-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-feb-2022/test.csv')

In [ ]:
print(train.shape)
cols = [e for e in test.columns if e not in ('row_id')]
train.drop_duplicates(subset=cols, keep='first',inplace = True)
print(train.shape)

In [ ]:
s1 = pd.merge(train, test, how='inner', on=cols)
s1

In [ ]:
s1.row_id_y.nunique()

In [ ]:
dic = {}
for i in range(len(s1)):
    dic[s1.loc[i]['row_id_y']] = s1.loc[i]['row_id_x']

In [ ]:
len(dic)

In [ ]:
for e in dic:
    sub.loc[sub[sub['row_id']==e].index.to_list(),'target'] = train.loc[train[train['row_id']==dic[e]].index.tolist()[0],'target']

In [ ]:
sub.to_csv("submission.csv", index=False)
sub

In [ ]:
sub.target.value_counts()